In [3]:
try:
    import GetOldTweets3 as got
except:
    !pip install GetOldTweets3
    import GetOldTweets3 as got

In [4]:
try:
    from bs4 import BeautifulSoup
except:
    !pip install BeautifulSoup
    from bs4 import BeautifulSoup

In [5]:
from random import uniform
from tqdm import tqdm_notebook
import pandas as pd
import re
import string

In [6]:
import datetime

days_range = []

#수집 기간 설정
start = datetime.datetime.strptime("2020-08-25", "%Y-%m-%d")
end = datetime.datetime.strptime("2020-08-26", "%Y-%m-%d")
date_generated = [start + datetime.timedelta(days=x) for x in range(0, (end-start).days)]

for date in date_generated:
    days_range.append(date.strftime("%Y-%m-%d"))
    
print("=== 설정된 트윗 수집 기간은 {} 에서 {}까지 입니다. ===".format(days_range[0], days_range[-1]))
print("=== 총 {}일 간의 데이터 수집 ===".format(len(days_range)))

=== 설정된 트윗 수집 기간은 2020-08-25 에서 2020-08-25까지 입니다. ===
=== 총 1일 간의 데이터 수집 ===


In [8]:
import time

start_date = days_range[0]
end_date = (datetime.datetime.strptime(days_range[-1], "%Y-%m-%d") 
            + datetime.timedelta(days=1)).strftime("%Y-%m-%d")

# 검색어 = #슬픔, #극혐, #무섭, 기쁨 
# '극혐', '슬픔', '공포', '기쁨' 시신 or 살인 or 납치 or 유괴 or 변사체
# 공포 정서에 공포영화를 넣어보는 것도 좋을 듯
# 공포 - 뱀, 바퀴벌레, 바선생, 사이비, 공포증, 뱀, 거미, 악천후, 벌레
# 5일치 - ㄱㅇㄷ:828, 무지개다리: 102, 딥빡: 367
# 10일치 - 개빡: 1953
#무지개다리 별로 -> 너무 적음, 유기견, 안락사로 대체
search_word_list = ['태풍']
for search_word in search_word_list:
    print(search_word)
    tweetCriteria = got.manager.TweetCriteria().setQuerySearch(search_word).setSince(start_date).setUntil(end_date).setMaxTweets(-1)
    print("Collecting data start.. from {} to {}".format(days_range[0], days_range[-1]))
    start_time = time.time()

    tweet = got.manager.TweetManager.getTweets(tweetCriteria)

    print("Collecting data end.. {0:0.2f} Minutes".format((time.time() - start_time)/60))
    print("=== Total num of tweets is {} ===".format(len(tweet)))
    tweet_list = []


    r = re.compile('#\S+|^https?\S+|@\S+')
    hangul = re.compile('[^ ㄱ-ㅣ가-힣|0-9|!$%&’()*+,.;<=>?\^_`{|}~-]+')
    for index in tqdm_notebook(tweet):
        content = index.text
        content = hangul.sub('',content)
        content = re.sub(r, '', content)
        content = re.sub('  +','',content)
        if content != '' and content != ' ':
            tweet_list.append(content)
    
    twitter_df = pd.DataFrame(tweet_list, columns = ['raw_text'])

    twitter_df.to_excel("tweet_list_{}.xlsx".format(search_word))
    print("=== {} tweets are successfully saved ===".format(len(tweet_list)))


태풍
=== Total num of tweets is 14011 ===


C:\anaconda\envs\pytorch\lib\site-packages\ipykernel_launcher.py:30: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



=== 13997 tweets are successfully saved ===


In [4]:
df_tweet = pd.read_excel("tweet_list.xlsx")
df_tweet.head(10)

NameError: name 'pd' is not defined